In [1]:
import pandas as pd

movies_raw = pd.read_csv('../movie-data.csv')
users_raw = pd.read_csv('../users-data.csv')
ratings_raw = pd.read_csv('../ratings-data.csv')

users_raw.set_index('user_id', inplace=True)
movies_raw.set_index('movie_id', inplace=True)

In [2]:
movies_raw.head()

,budget,genres,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title,tmdb_id,vote_average,vote_count
movie_id,,,,,,,,,,,,,,,,,
if+you+could+only+cook+1935,0.0,"['Comedy', 'Romance']",tt0026519,en,If You Could Only Cook,An auto engineer (Herbert Marshall) and a prof...,0.430813,['Columbia Pictures'],['United States of America'],1935-12-30,0.0,72,['English'],If You Could Only Cook,126083,6.8,4
the+tattooed+widow+1998,0.0,[],tt0120299,sv,Den tatuerade änkan,Ester is the perfect grandmother everyone expe...,0.012333,[],[],1998-03-10,0.0,0,[],The Tattooed Widow,80059,0.0,0
pride+and+prejudice+1980,0.0,['Drama'],tt0078672,en,Pride and Prejudice,Mrs. Bennet is determined to find husbands for...,0.073979,"['Australian Broadcasting Corporation', 'Briti...","['United Kingdom', 'Australia']",1980-01-13,0.0,265,['English'],Pride and Prejudice,77172,6.8,14
flodder+1986,0.0,['Comedy'],tt0091060,fr,Flodder,A low-class a-social family ends up in a rich ...,6.702840,['First Floor Features'],['Netherlands'],1986-12-17,0.0,111,['Nederlands'],Flodder,10570,6.6,32
cold+turkey+1971,0.0,['Comedy'],tt0066927,en,Cold Turkey,Reverend Brooks leads the town in a contest to...,0.609080,"['Tandem Productions', 'DFI']",['United States of America'],1971-02-19,11000000.0,99,['English'],Cold Turkey,42493,7.5,4


In [3]:
def transform_multilabel_binarizer_cols(df, col_options):
    binarizers = []
    sparse_df = df.copy()
    for col_option in col_options:
        col_name = col_option.get('name')
#         is_col_arraystr = col_option.get('is_col_arraystr', False)
#         is_single_char = col_option.get('is_single_char',False)
        new_col, mlb = create_multilabel_binarizer(sparse_df[col_name])
        
        binarizers.append((col_name,mlb))
        print('Column: {NAME} Values: {OPTIONS}'.format(NAME=col_name, OPTIONS=mlb.classes_))

        sparse_df = pd.concat([sparse_df, new_col],axis=1)
        sparse_df.drop(col_name, axis=1, inplace=True)
    
    return sparse_df, binarizers

def create_multilabel_binarizer(column):
    transformed_column = column.copy()

    transformed_column.loc[transformed_column.isnull()] = transformed_column.loc[
        transformed_column.isnull()
    ].apply(lambda x: '[]')
    transformed_column = transformed_column.apply(parse_arraystr)
#     elif is_single_char:
#         transformed_colum = transformed_column
#     else:
#         transformed_column = transformed_column
    
    mlb = MultiLabelBinarizer()
    mlb.fit(transformed_column)
    return pd.DataFrame(mlb.transform(transformed_column),columns=mlb.classes_), mlb

    
def parse_arraystr(str):
    str_without_brackets = str.replace("[","").replace("]","")
    str_without_quotes = str_without_brackets.replace("'","")
    str_without_spaces = str_without_quotes.replace(" ","")
    return np.array(str_without_spaces.split(','))

In [4]:
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import json

ratings = ratings_raw.join(users_raw, on='user_id', how='left')
ratings = ratings.join(movies_raw, on='movie_id', how='left')

ratings, binarizers = transform_multilabel_binarizer_cols(ratings, [
    {'name':'genres'},
    {'name':'production_countries'},
    {'name':'spoken_languages'},
    {'name':'gender'},
    {'name':'occupation'},
])

ratings = ratings.dropna(subset=['release_date'])
ratings['year'] = ratings['release_date'].apply(lambda x: str(x)[:4])
ratings['month'] = ratings['release_date'].apply(lambda x: str(x)[5:7])
ratings = ratings.astype({'year':'int64', 'month':'int64'})
ratings.drop('release_date', axis=1, inplace=True)

ratings = ratings.sort_values(by=['user_id'])
ratings.head()

Column: genres Values: ['' 'Action' 'Adventure' 'Animation' 'Comedy' 'Crime' 'Documentary'
 'Drama' 'Family' 'Fantasy' 'Foreign' 'History' 'Horror' 'Music' 'Mystery'
 'Romance' 'ScienceFiction' 'TVMovie' 'Thriller' 'War' 'Western']
Column: production_countries Values: ['' '"CoteDIvoire"' 'Afghanistan' 'Algeria' 'Angola' 'Argentina' 'Armenia'
 'Aruba' 'Australia' 'Austria' 'Bahamas' 'Belgium' 'Bhutan'
 'BosniaandHerzegovina' 'Botswana' 'Brazil' 'Bulgaria' 'BurkinaFaso'
 'Cameroon' 'Canada' 'Chile' 'China' 'Colombia' 'Congo' 'CostaRica'
 'Croatia' 'Cuba' 'Cyprus' 'CzechRepublic' 'Czechoslovakia' 'Denmark'
 'DominicanRepublic' 'Ecuador' 'Egypt' 'Estonia' 'Finland' 'France'
 'Georgia' 'Germany' 'Greece' 'HongKong' 'Hungary' 'Iceland' 'India'
 'Indonesia' 'Iran' 'Iraq' 'Ireland' 'Israel' 'Italy' 'Jamaica' 'Japan'
 'Kazakhstan' 'Latvia' 'Lebanon' 'LibyanArabJamahiriya' 'Liechtenstein'
 'Lithuania' 'Luxembourg' 'Macedonia' 'Malaysia' 'Malta' 'Mexico' 'Monaco'
 'Mongolia' 'Morocco' 'Namibia' '

,movie_id,rating,user_id,age,budget,imdb_id,original_language,original_title,overview,popularity,...,retired,sales/marketing,scientist,self-employed,technician/engineer,tradesman/craftsman,unemployed,writer,year,month
18519,sleepless+in+seattle+1993,4,8,30,21000000.0,tt0108160,en,Sleepless in Seattle,A young boy who tries to set his dad up on a d...,10.23490,...,0,0,0,0,0,0,0,0,1993,6
18943,star+trek+generations+1994,3,8,30,38000000.0,tt0111280,en,Star Trek: Generations,Captain Jean-Luc Picard and the crew of the En...,8.10571,...,0,0,0,0,0,0,0,0,1994,11
17113,pulp+fiction+1994,5,8,30,8000000.0,tt0110912,en,Pulp Fiction,"A burger-loving hit man, his philosophical par...",140.95000,...,0,0,0,0,0,0,0,0,1994,9
20250,johnny+mnemonic+1995,1,8,30,25000000.0,tt0113481,en,Johnny Mnemonic,"A data courier, literally carrying a data pack...",11.71590,...,0,0,0,0,0,0,0,0,1995,5
18869,miracle+on+34th+street+1994,4,8,30,0.0,tt0110527,en,Miracle on 34th Street,A little girl discovers dreams can come true i...,5.83162,...,0,0,0,0,0,0,0,0,1994,11


In [9]:
from tpot import TPOTRegressor


X_train = ratings.drop(['movie_id','imdb_id','tmdb_id','user_id','title','overview', 'original_language',# 'genres', 'production_countries', 'spoken_languages', 'gender','occupation',
                        'original_title','production_companies','rating'], axis=1).dropna()


X_train = X_train
y_train = ratings['rating']

pipeline_optimizer = TPOTRegressor(generations=10, population_size=10, verbosity=3, random_state=42,
                                   template='Selector-Transformer-Regressor', n_jobs=8,
                                   warm_start=True, periodic_checkpoint_folder='../tpot-intermediate-save-3/')
pipeline_optimizer.fit(X_train.head(10000),y_train.head(10000))

30 operators have been imported by TPOT.


_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required..
_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required..

Generation 1 - Current Pareto front scores:
Saving periodic pipeline from pareto front to ../tpot-intermediate-save-3/pipeline_gen_1_idx_0_2020.07.11_01-36-47.pySVR__C=10.0, LinearSVR__dual=True, LinearSVR__epsilon=0.1, LinearSVR__loss=squared_epsilon_insensitive, LinearSVR__tol=0.1)

Generation 2 - Current Pareto front scores:
Periodic pipeline was not saved, probably saved before...tile(input_matrix, SelectPercentile__percentile=27)), LinearSVR__C=10.0, LinearSVR__dual=True, LinearSVR__epsilon=0.1, LinearSVR__loss=squared_epsilon_insensitive, LinearSVR__tol=0.1)
_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 2 is required by Feature

TPOTRegressor(generations=10,
              log_file=<ipykernel.iostream.OutStream object at 0x10a49efd0>,
              n_jobs=8,
              periodic_checkpoint_folder='../tpot-intermediate-save-3/',
              population_size=10, random_state=42,
              template='Selector-Transformer-Regressor', verbosity=3,
              warm_start=True)

In [10]:
pipeline_optimizer.score(X_train[10001:20000], y_train[10001:20000])

-0.9845655731739391

In [13]:

pipeline_optimizer_nn = TPOTRegressor(generations=10, population_size=50, verbosity=3, random_state=42,
                                      template='Selector-Transformer-Regressor',config_dict="TPOT light",# n_jobs=4,
                                     warm_start=True, periodic_checkpoint_folder='../tpot-intermediate-save-2/')
pipeline_optimizer_nn.fit(X_train.head(500000),y_train.head(500000))

20 operators have been imported by TPOT.


Skipped pipeline #3 due to time out. Continuing to the next pipeline.
Skipped pipeline #9 due to time out. Continuing to the next pipeline.
Skipped pipeline #17 due to time out. Continuing to the next pipeline.
Skipped pipeline #20 due to time out. Continuing to the next pipeline.
Skipped pipeline #22 due to time out. Continuing to the next pipeline.
Skipped pipeline #31 due to time out. Continuing to the next pipeline.
Skipped pipeline #33 due to time out. Continuing to the next pipeline.
Skipped pipeline #38 due to time out. Continuing to the next pipeline.
Skipped pipeline #42 due to time out. Continuing to the next pipeline.
Skipped pipeline #45 due to time out. Continuing to the next pipeline.
Skipped pipeline #49 due to time out. Continuing to the next pipeline.
Skipped pipeline #57 due to time out. Continuing to the next pipeline.
_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required by MinMaxSca

TPOTRegressor(config_dict='TPOT light', generations=10,
              log_file=<ipykernel.iostream.OutStream object at 0x10fe7cd60>,
              periodic_checkpoint_folder='../tpot-intermediate-save-2/',
              population_size=50, random_state=42,
              template='Selector-Transformer-Regressor', verbosity=3,
              warm_start=True)

In [14]:
pipeline_optimizer_nn.score(X_train[500001:700000], y_train[500001:700000])

-0.9647899279603104